# style-transfer-pytorch

An implementation of neural style transfer ([A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576)) in PyTorch, supporting CPUs and Nvidia GPUs. It does automatic multi-scale (coarse-to-fine) stylization to produce high-quality high resolution stylizations, even up to print resolution if the GPUs have sufficient memory.

See [the GitHub repo](https://github.com/crowsonkb/style-transfer-pytorch) for more information.

In [ ]:
!pip install cloudinary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
!nvidia-smi
!git clone https://github.com/crowsonkb/style-transfer-pytorch
!pip install -e ./style-transfer-pytorch

Mon Mar 24 05:40:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Cloning into 'style-transfer-pytorch'...


remote: Enumerating objects: 567, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (28/28), done.


remote: Total 567 (delta 212), reused 206 (delta 197), pack-reused 342 (from 1)
Receiving objects: 100% (567/567), 2.72 MiB | 5.85 MiB/s, done.
Resolving deltas: 100% (361/361), done.


Obtaining file:///kaggle/working/style-transfer-pytorch


  Preparing metadata (setup.py) ... done


  Running setup.py develop for style-transfer-pytorch


In [ ]:
import json, random, subprocess, os#, cloudinary, cloudinary.uploader
from pathlib import Path

In [ ]:
%%writefile /kaggle/working/style-transfer-pytorch/style_transfer/cli.py

"""Neural style transfer (https://arxiv.org/abs/1508.06576) in PyTorch."""

import argparse
import atexit
from dataclasses import asdict
import io, os, uuid
import json, base64
from pathlib import Path
import platform
import sys
import webbrowser
import requests
from typing import Union

import numpy as np
from PIL import Image, ImageCms
from tifffile import TIFF, TiffWriter
import torch
import torch.multiprocessing as mp
from tqdm import tqdm

import cloudinary
import cloudinary.uploader
from cloudinary.utils import cloudinary_url

# Configuration
cloudinary.config(
    cloud_name = "<cloud_name>",
    api_key = "<api_key>",
    api_secret = "<api_secret>", # Click 'View API Keys' above to copy your API secret
    secure=True
)

from . import srgb_profile, StyleTransfer, WebInterface

DEFAULT_RETURN_TYPE = 'response'
def _pantry_call(pantry_id: str, request_type: str, name: str=None, contents: dict=None, return_type: str=DEFAULT_RETURN_TYPE) -> Union[str, dict]:
	"""private base function for other API call functions"""

	url = f"https://getpantry.cloud/apiv1/pantry/{pantry_id}"
	if name != None and name.strip() != '':
		# if a basket name is passed, it should be added to the URL
		url += f"/basket/{name}"

	# if data is passed as 'contents', convert it to string to add to the request, else leave it empty
	if contents != None:
		payload = json.dumps(contents)
	else:
		payload = ""

	headers= {
		'Content-Type': 'application/json'
	}
	# request_type is passed by the individual wrapper functions;
	# the Pantry API uses the same-ish URL endpoint but different HTTP methods for the different functions
	response = requests.request(request_type, url, headers=headers, data=payload)

	# two options to return: the whole Response object, or just the data from the body (string or JSON)
	if return_type.strip().lower() == 'response':
		return response
	elif return_type.strip().lower() == 'body':
		return _format_body(response.text)
	else:
		return None

def append_basket(pantry_id: str, name: str, contents: dict, return_type: str=DEFAULT_RETURN_TYPE) -> Union[str, dict]:
	"""appends contents to new or existing basket; will overwrite values of existing keys and append values to nested objects or arrays"""
	return _pantry_call(pantry_id, request_type='PUT', name=name, contents=contents, return_type=return_type)


def prof_to_prof(image, src_prof, dst_prof, **kwargs):
    src_prof = io.BytesIO(src_prof)
    dst_prof = io.BytesIO(dst_prof)
    return ImageCms.profileToProfile(image, src_prof, dst_prof, **kwargs)


def load_image(url=None, path=None, proof_prof=None):
    src_prof = dst_prof = srgb_profile
    try:
      if url:
        response = requests.get(url)
        image = Image.open(io.BytesIO(response.content))
      elif path:
        image = Image.open(path)

      if 'icc_profile' in image.info:
          src_prof = image.info['icc_profile']
      else:
          image = image.convert('RGB')
      if proof_prof is None:
          if src_prof == dst_prof:
              return image.convert('RGB')
          return prof_to_prof(image, src_prof, dst_prof, outputMode='RGB')
      proof_prof = Path(proof_prof).read_bytes()
      cmyk = prof_to_prof(image, src_prof, proof_prof, outputMode='CMYK')
      return prof_to_prof(cmyk, proof_prof, dst_prof, outputMode='RGB')

    except OSError as err:
        print_error(err)
        sys.exit(1)


def save_pil(path, image):
    try:
        kwargs = {'icc_profile': srgb_profile}
        if path.suffix.lower() in {'.jpg', '.jpeg'}:
            kwargs['quality'] = 95
            kwargs['subsampling'] = 0
        elif path.suffix.lower() == '.webp':
            kwargs['quality'] = 95
        image.save(path, **kwargs)
    except (OSError, ValueError) as err:
        print_error(err)
        sys.exit(1)


def save_tiff(path, image):
    tag = ('InterColorProfile', TIFF.DATATYPES.BYTE, len(srgb_profile), srgb_profile, False)
    try:
        with TiffWriter(path) as writer:
            writer.save(image, photometric='rgb', resolution=(72, 72), extratags=[tag])
    except OSError as err:
        print_error(err)
        sys.exit(1)


def save_image(path, image, iterations_images):
    unique_id = str(uuid.uuid4()).replace("-", "")
    i = str(len(iterations_images))
    pathlib_path = Path(i + unique_id + path)
    tqdm.write(f'Writing image to {path}.')
    if isinstance(image, Image.Image):
        save_pil(pathlib_path, image)

        cloudinary_url = image_upload_get_url(pathlib_path, image)

        iterations_images.append(cloudinary_url)

    elif isinstance(image, np.ndarray) and path.suffix.lower() in {'.tif', '.tiff'}:
        save_tiff(path, image)
    else:
        raise ValueError('Unsupported combination of image type and extension')


def image_upload_get_url(path, image=None):
    pathlib_path = Path(path)

    upload_result = cloudinary.uploader.upload(path,
                                           public_id=pathlib_path.stem)
    return upload_result["secure_url"]


def write_to_json(res_url, content_path, style_path, option3_path, option4_path, iteration_image_urls):
    content_url = image_upload_get_url(content_path)
    style_url = image_upload_get_url(style_path[0])
    option3_url = image_upload_get_url(option3_path)
    option4_url = image_upload_get_url(option4_path)

    data = {res_url: [[content_url, style_url, option3_url, option4_url], [iteration_image_urls]]}

    append_basket("<basket_id>", "<basket_name>", data, return_type='body')


def get_safe_scale(w, h, dim):
    """Given a w x h content image and that a dim x dim square does not
    exceed GPU memory, compute a safe end_scale for that content image."""
    return int(pow(w / h if w > h else h / w, 1/2) * dim)


def setup_exceptions():
    try:
        from IPython.core.ultratb import FormattedTB
        sys.excepthook = FormattedTB(mode='Plain', color_scheme='Neutral')
    except ImportError:
        pass


def fix_start_method():
    if platform.system() == 'Darwin':
        mp.set_start_method('spawn')


def print_error(err):
    print('\033[31m{}:\033[0m {}'.format(type(err).__name__, err), file=sys.stderr)


class Callback:
    def __init__(self, st, args, image_type='pil', web_interface=None):
        self.st = st
        self.args = args
        self.image_type = image_type
        self.web_interface = web_interface
        self.iterates = []
        self.progress = None
        self.iteration_image_urls = []

    def __call__(self, iterate):
        self.iterates.append(asdict(iterate))
        if iterate.i == 1:
            self.progress = tqdm(total=iterate.i_max, dynamic_ncols=True)
        msg = 'Size: {}x{}, iteration: {}, loss: {:g}'
        tqdm.write(msg.format(iterate.w, iterate.h, iterate.i, iterate.loss))
        self.progress.update()
        if self.web_interface is not None:
            self.web_interface.put_iterate(iterate, self.st.get_image_tensor())
        if iterate.i == iterate.i_max:
            self.progress.close()

            if max(iterate.w, iterate.h) != self.args.end_scale:
                save_image(self.args.output, self.st.get_image(self.image_type), self.iteration_image_urls)

            else:
                if self.web_interface is not None:
                    self.web_interface.put_done()
        elif iterate.i % self.args.save_every == 0:
            save_image(self.args.output, self.st.get_image(self.image_type), self.iteration_image_urls)

    def close(self):
        if self.progress is not None:
            self.progress.close()

    def get_trace(self):
        return {'args': self.args.__dict__, 'iterates': self.iterates}


def main():
    setup_exceptions()
    fix_start_method()

    p = argparse.ArgumentParser(description=__doc__,
                                formatter_class=argparse.ArgumentDefaultsHelpFormatter)

    def arg_info(arg):
        defaults = StyleTransfer.stylize.__kwdefaults__
        default_types = StyleTransfer.stylize.__annotations__
        return {'default': defaults[arg], 'type': default_types[arg]}

    p.add_argument('content', type=str, help='the content image')
    p.add_argument('styles', type=str, nargs='+', metavar='style', help='the style images')
    p.add_argument('--output', '-o', type=str, default='out.png',
                   help='the output image')
    p.add_argument('--option3', type=str, help='random image to be displayed')
    p.add_argument('--option4', type=str, help='another random image to be displayed')
    p.add_argument('--style-weights', '-sw', type=float, nargs='+', default=None,
                   metavar='STYLE_WEIGHT', help='the relative weights for each style image')
    p.add_argument('--devices', type=str, default=[], nargs='+',
                   help='the device names to use (omit for auto)')
    p.add_argument('--random-seed', '-r', type=int, default=0,
                   help='the random seed')
    p.add_argument('--content-weight', '-cw', **arg_info('content_weight'),
                   help='the content weight')
    p.add_argument('--tv-weight', '-tw', **arg_info('tv_weight'),
                   help='the smoothing weight')
    p.add_argument('--optimizer', **arg_info('optimizer'),
                   choices=['adam', 'lbfgs'],
                   help='the optimizer to use')
    p.add_argument('--min-scale', '-ms', **arg_info('min_scale'),
                   help='the minimum scale (max image dim), in pixels')
    p.add_argument('--end-scale', '-s', type=str, default='512',
                   help='the final scale (max image dim), in pixels')
    p.add_argument('--iterations', '-i', **arg_info('iterations'),
                   help='the number of iterations per scale')
    p.add_argument('--initial-iterations', '-ii', **arg_info('initial_iterations'),
                   help='the number of iterations on the first scale')
    p.add_argument('--save-every', type=int, default=50,
                   help='save the image every SAVE_EVERY iterations')
    p.add_argument('--step-size', '-ss', **arg_info('step_size'),
                   help='the step size (learning rate) for Adam')
    p.add_argument('--avg-decay', '-ad', **arg_info('avg_decay'),
                   help='the EMA decay rate for iterate averaging')
    p.add_argument('--init', **arg_info('init'),
                   choices=['content', 'gray', 'uniform', 'normal', 'style_stats'],
                   help='the initial image')
    p.add_argument('--style-scale-fac', **arg_info('style_scale_fac'),
                   help='the relative scale of the style to the content')
    p.add_argument('--style-size', **arg_info('style_size'),
                   help='the fixed scale of the style at different content scales')
    p.add_argument('--pooling', type=str, default='max', choices=['max', 'average', 'l2'],
                   help='the model\'s pooling mode')
    p.add_argument('--proof', type=str, default=None,
                   help='the ICC color profile (CMYK) for soft proofing the content and styles')
    p.add_argument('--web', default=False, action='store_true', help='enable the web interface')
    p.add_argument('--host', type=str, default='0.0.0.0',
                   help='the host the web interface binds to')
    p.add_argument('--port', type=int, default=8080,
                   help='the port the web interface binds to')
    p.add_argument('--browser', type=str, default='', nargs='?',
                   help='open a web browser (specify the browser if not system default)')

    args = p.parse_args()

    content_img = load_image(path=args.content, proof_prof=args.proof)
    style_imgs = [load_image(path=img, proof_prof=args.proof) for img in args.styles]

    image_type = 'pil'
    if Path(args.output).suffix.lower() in {'.tif', '.tiff'}:
        image_type = 'np_uint16'

    devices = [torch.device(device) for device in args.devices]
    if not devices:
        devices = [torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')]
    if len(set(device.type for device in devices)) != 1:
        print('Devices must all be the same type.')
        sys.exit(1)
    if not 1 <= len(devices) <= 2:
        print('Only 1 or 2 devices are supported.')
        sys.exit(1)
    print('Using devices:', ' '.join(str(device) for device in devices))

    if devices[0].type == 'cpu':
        print('CPU threads:', torch.get_num_threads())
    if devices[0].type == 'cuda':
        for i, device in enumerate(devices):
            props = torch.cuda.get_device_properties(device)
            print(f'GPU {i} type: {props.name} (compute {props.major}.{props.minor})')
            print(f'GPU {i} RAM:', round(props.total_memory / 1024 / 1024), 'MB')

    end_scale = int(args.end_scale.rstrip('+'))
    if args.end_scale.endswith('+'):
        end_scale = get_safe_scale(*content_img.size, end_scale)
    args.end_scale = end_scale

    web_interface = None
    if args.web:
        web_interface = WebInterface(args.host, args.port)
        atexit.register(web_interface.close)

    for device in devices:
        torch.tensor(0).to(device)
    torch.manual_seed(args.random_seed)

    print('Loading model...')
    st = StyleTransfer(devices=devices, pooling=args.pooling)
    callback = Callback(st, args, image_type=image_type, web_interface=web_interface)
    atexit.register(callback.close)

    url = f'http://{args.host}:{args.port}/'
    if args.web:
        if args.browser:
            webbrowser.get(args.browser).open(url)
        elif args.browser is None:
            webbrowser.open(url)

    defaults = StyleTransfer.stylize.__kwdefaults__
    st_kwargs = {k: v for k, v in args.__dict__.items() if k in defaults}
    try:
        st.stylize(content_img, style_imgs, **st_kwargs, callback=callback)
    except KeyboardInterrupt:
        pass

    output_image = st.get_image(image_type)
    if output_image is not None:
        save_image(args.output, output_image, callback.iteration_image_urls)
        write_to_json(callback.iteration_image_urls[-1], args.content, args.styles,
                      args.option3, args.option4, callback.iteration_image_urls)
    with open('trace.json', 'w') as fp:
        json.dump(callback.get_trace(), fp, indent=4)


if __name__ == '__main__':
    main()

Overwriting /kaggle/working/style-transfer-pytorch/style_transfer/cli.py


In [ ]:
def random_image_from_directory(directory, seen):
    images = [os.path.join(directory, file) for file in sorted(os.listdir(directory))]
    img_path = random.choice(images)
    while img_path in seen:
      img_path = random.choice(images)
    return img_path

def choose_4_random_images_and_return_list():
  # CHOOSE CATEGORIES FOR IMAGES
  categories = ['/kaggle/input/images-dataset/ST_data/cities_square/',
                '/kaggle/input/images-dataset/ST_data/city_ai_photos',
                '/kaggle/input/images-dataset/ST_data/landscape_paintings/',
                '/kaggle/input/images-dataset/ST_data/nature_views/',
                '/kaggle/input/images-dataset/ST_data/paintings/']

  cat_content = random.choice(categories)
  cat_style = random.choice(categories)
  cat_random_1 = random.choice(categories)
  cat_random_2 = random.choice(categories)

  while cat_content == cat_style:
    cat_style = random.choice(categories)


  # CHOOSE IMAGES
  seen = []

  path_content = random_image_from_directory(cat_content, seen)

  path_style = random_image_from_directory(cat_style, seen)

  path_random_1 = random_image_from_directory(cat_random_1, seen)

  path_random_2 = random_image_from_directory(cat_random_2, seen)

  seen.append(path_content)
  seen.append(path_style)
  seen.append(path_random_1)
  seen.append(path_random_2)

  return seen

In [ ]:
import os, uuid

image_dict_content = []
for i in range(150):
    imgs = choose_4_random_images_and_return_list()
    image_dict_content.append(imgs)

In [ ]:
for i in range(len(image_dict_content)):
    images = image_dict_content[i]
    content_url = images[0]
    style_url = images[1]
    random1_url = images[2]
    random2_url = images[3]
    # Define output file path based on iteration index
    i = str(uuid.uuid4()).replace("-", "")
    output_path = f"out{i}.png"

    # Build the command for subprocess
    command = [
        "style_transfer",
        content_url,
        style_url,
        "--output",
        output_path,
        "--option3",
        random1_url,
        "--option4",
        random2_url,
        "-s",
        "512",
        "-ms",
        "512",
        "--save-every",
        "10"
    ]

    # Execute the command
    try:
        with open("output.log", "a") as log_file:
            subprocess.run(command, stdout=log_file, stderr=log_file)
        print("Successful: ")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {output_image_url}: {e}")

# cmd = " ".join(command)
# !{cmd}

Successful: 


Successful: 


Successful: 


Successful: 


Successful: 


...

Successful: 


Successful: 


Successful: 


Successful: 


Successful: 
